In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.head()

#  embbeding

In [ ]:
import gensim.downloader as api
wvv=api.load('word2vec-google-news-300')

In [ ]:
wvv.most_similar('good')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
def preprocessing_and_vectorize(text):
    
    doc=nlp(text)
    
    filtered_tokens=[]
    for token in doc:
        if token.is_punct or token.is_stop:
            continue
        filtered_tokens.append(token.lemma_)
    return wvv.get_mean_vector(filtered_tokens)# want sentance embedding

In [ ]:
#create a new columns and vector as sentence enbedding

df['vector']=df['text'].apply(lambda text:preprocessing_and_vectorize(text))

In [ ]:
df.head()

### train test split

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test = train_test_split(
#     df.vector.values,
#     df.target,
#     test_size=0.2,
#     random_state=2022,
#     stratify=df.target
# )

In [ ]:
X=df.vector.values

In [ ]:
y=df.target

### Reshaping the X_train and X_test so as to fit for models

In [ ]:
print(" Shape of x_train before reshaping :",X.shape)


X_train_2d = np.stack(X)

print('Shape of X_train after reshaping: ', X_train_2d.shape)


### pepare test data

In [ ]:
df_test=pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
df_test.head()

In [ ]:
df_test.isnull().sum()

In [ ]:
def get_word_vector(word):
    try:
        # Try to get the word vector
        return model.wv[word]
    except KeyError:
        # If the word is not in the vocabulary, return a zero vector
        return np.zeros(vector_size)

In [ ]:
def preprocessing_and_vectorize2(text):
    
    doc=nlp(text)
    
    filtered_tokens=[]
    for token in doc:
        if token.is_punct or token.is_stop:
            continue
        filtered_tokens.append(token.lemma_)
    try:
        return wvv.get_mean_vector(filtered_tokens)# want sentance embedding
    except ValueError:
        return np.zeros(300)

In [ ]:
df_test['vector']=df_test['text'].apply(lambda text:preprocessing_and_vectorize2(text))

In [ ]:
df_test.head()

In [ ]:
X_test=df_test.vector.values

In [ ]:
print(" Shape of x_train before reshaping :",X_test.shape)


X_test_2d = np.stack(X_test)

print('Shape of X_train after reshaping: ', X_test_2d.shape)

## traning model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
svc_clf=SVC()
sgd_clf=SGDClassifier()
tree_clf=DecisionTreeClassifier()
rnd_clf=RandomForestClassifier()
log_clf=LogisticRegression()

In [ ]:
from sklearn.model_selection import cross_val_score

models=[svc_clf,sgd_clf,tree_clf,rnd_clf,log_clf]
for model in models:
   acc=cross_val_score(model, X_train_2d,y,cv=3,scoring="accuracy")
   print(f'model:{model}---->{acc}')

In [ ]:
from sklearn.svm import SVC

svc_clf=SVC()

svc_clf.fit(X_train_2d,y)


y_pred_svc=svc_clf.predict(X_test_2d)

In [ ]:
y_pred_svc

# voting classifiers

In [ ]:
# from sklearn.ensemble import VotingClassifier

In [ ]:
# voting_clf=VotingClassifier(estimators=[('lr',log_clf),('',sgd_clf)],voting='soft')


In [ ]:
# cross_val_score(voting_clf, X_train_2d,y,cv=3,scoring="accuracy")

In [ ]:
# voting_clf.fit(X_train_2d,y)
# y_pred_voting=voting_clf.predict(X_test)

### submission

In [ ]:
df_sub=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [ ]:
df_sub.head()

In [ ]:
output = pd.DataFrame({'id': df_sub.id, 'target': y_pred_svc})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

## huggingface

In [ ]:
import pandas as pd
df=pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
df

In [ ]:
df.text

In [ ]:
text=df.iloc[1,3]

In [ ]:
text

In [ ]:
from transformers import pipeline
classifier=pipeline('text-classification')



In [ ]:

outputs=classifier(text)
pd.DataFrame(outputs)